In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import seaborn as sns

orders = pd.read_csv(r'C:\Users\DELL\Downloads\orders.csv')
returns = pd.read_csv(r'C:\Users\DELL\Downloads\returns.csv')
products = pd.read_csv(r'C:\Users\DELL\Downloads\product_master.csv')
customers = pd.read_csv(r'C:\Users\DELL\Downloads\customer_master.csv')
stores = pd.read_csv(r'C:\Users\DELL\Downloads\store_master.csv')


In [3]:
print(orders.head())

   OrderID ProductID CustomerID StoreID  Quantity       GMV  Revenue
0  O000001     P0288     C04812    S151         3  11204.69  9523.99
1  O000002     P0395     C04351    S141         4    453.03   385.08
2  O000003     P0339     C02534    S183         1   1126.15   957.22
3  O000004     P0358     C00476    S076         3   4819.73  4096.77
4  O000005     P0388     C01148    S024         1   4766.82  4051.80


In [4]:
print(orders.describe())

           Quantity           GMV       Revenue
count  20000.000000  20000.000000  20000.000000
mean       2.506500   6388.266237   5430.026293
std        1.116835   4819.006712   4096.155692
min        1.000000    100.880000     85.750000
25%        2.000000   2556.235000   2172.795000
50%        3.000000   4898.165000   4163.440000
75%        4.000000   9306.427500   7910.462500
max        4.000000  19995.950000  16996.550000


In [5]:
print(orders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   OrderID     20000 non-null  object 
 1   ProductID   20000 non-null  object 
 2   CustomerID  20000 non-null  object 
 3   StoreID     20000 non-null  object 
 4   Quantity    20000 non-null  int64  
 5   GMV         20000 non-null  float64
 6   Revenue     20000 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.1+ MB
None


2. Check data types, missing values, duplicates, unique values:

In [6]:
orders.isnull().sum()

OrderID       0
ProductID     0
CustomerID    0
StoreID       0
Quantity      0
GMV           0
Revenue       0
dtype: int64

In [7]:
orders.duplicated().sum()

np.int64(0)

Merge all datasets to get a one dataset (sales):

In [8]:
# Merge Orders with Product, Customer, Store info
df = orders.merge(products, on='ProductID', how='left') \
           .merge(customers, on='CustomerID', how='left') \
           .merge(stores, on='StoreID', how='left')

In [9]:
# Merge returns if needed for return analysis
df = df.merge(returns[['OrderID', 'Reason']], on='OrderID', how='left')

In [10]:
print(df.columns)

Index(['OrderID', 'ProductID', 'CustomerID', 'StoreID', 'Quantity', 'GMV',
       'Revenue', 'ProductName', 'Category', 'Brand', 'Price', 'CustomerName',
       'City_x', 'State', 'StoreName', 'City_y', 'Reason'],
      dtype='object')


#### Step 3: Data Cleaning

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df['Reason'] = df['Reason'].fillna('No Return')

In [13]:
df['Margin'] = df['Revenue'] - df['GMV'] 
df['ReturnFlag'] = df['Reason'].apply(lambda x: 0 if x == 'No Return' else 1)

#### Step 4: KPI

In [14]:
total_sales = df['Revenue'].sum()
total_gmv = df['GMV'].sum()
total_margin = df['Margin'].sum()
avg_aov = df['Revenue'].sum() / df['OrderID'].nunique()
total_units = df['Quantity'].sum()
return_rate = df['ReturnFlag'].mean() * 100
repeat_rate = (df.groupby('CustomerID')['OrderID'].nunique() > 1).mean() * 100